In [9]:
import glob
import json
import os
import pickle
import pickle as pkl

import hist as hist2
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import yaml
from hist.intervals import clopper_pearson_interval
from scipy.special import softmax
from sklearn.metrics import auc, roc_curve

pd.options.mode.chained_assignment = None

plt.rcParams.update({"font.size": 20})
plt.style.use(hep.style.CMS)

In [16]:
! ls /Users/fmokhtar/Desktop/AN_2024/ele_MVA_test/

0-1.pkl   0-10.pkl  0-100.pkl


In [17]:
ch = "ele"

with open("/Users/fmokhtar/Desktop/AN_2024/ele_MVA_test/0-100.pkl", "rb") as f:
    file = pkl.load(f)
    
    trg_dict = file["2017"]["GluGluHToWW_Pt-200ToInf_M-125"]["skimmed_events"][ch]["triggers"]
    var_dict = file["2017"]["GluGluHToWW_Pt-200ToInf_M-125"]["skimmed_events"][ch]["vars"]
    weight_dict = file["2017"]["GluGluHToWW_Pt-200ToInf_M-125"]["skimmed_events"][ch]["weights"]

var_dict["fj_genH_pt"] = var_dict["fj_genH_pt"].reshape(-1)

In [18]:
nev = len(var_dict["fj_genH_pt"])
print("Events before trigger: ", nev)

########## build the OR of all triggers
msk_trg = np.zeros(len(var_dict["fj_genH_pt"]), dtype="bool")

for trig in trg_dict:
    msk_trg = msk_trg | trg_dict[trig]

print("Events after trigger: ", msk_trg.sum())

print("----------------------")
print("Trigger eff: ", msk_trg.sum()/nev)

Events before trigger:  7496
Events after trigger:  4749
----------------------
Trigger eff:  0.633537886872999


In [21]:
msk_trg.sum()

4749

In [22]:
(msk_trg & var_dict["ele_MVA"]).sum()

4248

In [23]:
100 * (msk_trg & var_dict["ele_MVA"]).sum() / msk_trg.sum()

89.45041061276058